## Constructing a supercell from a .xyz file from C2DB

In [5]:
# Import the necessary class from the script
import sys
sys.path.append('../scripts')
from xyz2supercell import StructureBuilder

In [6]:
filename = input("Enter /path/to/AB.xyz")

# Create an instance of the StructureBuilder class
builder = StructureBuilder(filename)

# Extract elements from the XYZ file
builder.extract_elements()

# Extract lattice constants from the XYZ file
builder.set_lattice_constants()

# Construct coordinates from the XYZ file
builder.construct_coordinates()

# Build the structure
builder.build_structure()

# Create a supercell by specifying the dimensions (e.g., 2x2x2)
scell_a = 2
scell_b = 2
scell_c = 1

builder.supercell(scell_a, scell_b, scell_c)

# Convert the structure to POSCAR format
builder.convert_to_poscar()

# Write the POSCAR to a file
builder.write_poscar_to_file()


Enter /path/to/AB.xyz ../structures/MoS2.xyz


## Generating a simple single atom vacancy using simple_defect_generation from a POSCAR file

In [7]:
# Import the necessary class from the script
import sys
sys.path.append('../scripts')
from simple_defect_generation import DefectFormation

filename = input("Enter POSCAR to defect (/path/to/AB.vasp)")

#create an instance of DefectFormation Class

defect_maker = DefectFormation(filename)

site_index = int(input("Enter site of defect: "))

vacancy_def = defect_maker.create_vacancy(site_index)

output_filename = input("Save defect POSCAR as: ")
defect_maker.write_poscar(output_filename)

Enter POSCAR to defect (/path/to/AB.vasp) ../structures/MoS2_2x2x1.vasp
Enter site of defect:  4
Save defect POSCAR as:  ../test/MoS2_2x2_V_S.vasp


At this point, we have a method to generating a POSCAR file given a pristine monolayer .xyz file from C2DB (since lattice constants are pulled from the first line of the .xyz file). To calculate a vasp calculation, a method of POTCAR, INCAR, KPOINT generation is necessary (also a jobscript if running on HPC)

In [31]:
# Generates a POTCAR file given a POSCAR

from pymatgen.io.vasp.inputs import Potcar
from pymatgen.core import Lattice, Structure, Molecule
from pathlib import Path

# Create a Structure object by reading your crystal structure from a file
structure = Structure.from_file("../test/MoS2_2x2_V_S.vasp")

# Initialize an empty list to store unique elements while preserving order
elements = []

# Iterate through the sites in the structure
for site in structure.sites:
    element = str(site.specie)
    if element not in elements:
        elements.append(element)

print(elements)

# Specify the functional choice
functional = "PBE"

# Create the Potcar object using the elements from the structure and the functional
potcar = Potcar(symbols=elements, functional=functional)

# Working directory
working_directory = Path("../test")
potcar_filepath = working_directory / "POTCAR"
potcar.write_file(potcar_filepath)
print("POTCAR generated")

['Mo', 'S']
POTCAR generated


In [33]:
# INCAR Generation
from pymatgen.io.vasp.inputs import Incar

# INCAR Parameters
incar_params = {
    "SYSTEM": "Your System Name",
    "ISTART": 0,
    "ICHARG": 2,
    "ENCUT": 400,
    "EDIFF": 1e-6,
    "NELM": 300,
    "ISMEAR": 0,
    "SIGMA": 0.05,
    "IALGO": 38,
    "ISIF": 4,
    "IBRION": 2,
    "NSW": 300,
    "EDIFFG": -0.005,
    "LCHARG": False,
    "LWAVE": False,
}

# Create an Incar object
incar = Incar(incar_params)

incar_filepath = working_directory / "INCAR"

# Write the INCAR file to the current working directory
incar.write_file(incar_filepath)


In [38]:
# KPOINT generation
from pymatgen.io.vasp.inputs import Kpoints

kpoints = Kpoints.monkhorst_automatic(kpts=(5, 5, 1))

# Write the KPOINTS file to the specified directory
kpoints.write_file(working_directory / "KPOINTS")
